# <strong>Hyperparameter Tuning</strong>

In [ ]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [ ]:
np.random.seed(12345)

In [ ]:
df_diab = pd.read_csv('diabetes.csv')

In [ ]:
df_diab.head()

In [ ]:
df_diab.Outcome.unique()

In [ ]:
df_diab.shape

In [ ]:
df_diab.isna().sum()

In [ ]:
X = df_diab.drop(['Outcome'],axis=1)
y = df_diab[['Outcome']]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train,y_test = train_test_split(X,
                                                   y,
                                                   stratify=y,
                                                   test_size=0.2,
                                                   random_state=12345)

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.models import Sequential

In [ ]:
from tensorflow.keras.layers import Dense, Dropout

### Hyperparameter Tuning


In [ ]:
!pip install -q -U keras-tuner

In [ ]:
def build_model(hp):
    # Instantiate the model
    model = Sequential()

    # Define first layer -> optimization for number of units and activation function
    model.add(Dense(units = hp.Int("units1",min_value = 32, max_value=1024, step=16),
                   activation = hp.Choice("activation1",["relu","tanh"]),
                    input_shape = (X_train_scaled.shape[1],)
                   ))

    # Define dropout layer by providing the optimization of dropout rate
    model.add(Dropout(hp.Float("rate1",min_value=0.1, max_value = 0.4, step=0.1),seed = 12345))

    # Add second layer
    model.add(Dense(units = hp.Int("units2",min_value = 32, max_value=128, step=16),
                   activation = hp.Choice("activation2",["relu","tanh"])
                   ))


    model.add(Dropout(hp.Float("rate2",min_value=0.1, max_value = 0.4, step=0.1),seed = 12345))

    model.add(Dense(units = hp.Int("units3",min_value = 32, max_value=128, step=16),
                   activation = hp.Choice("activation3",["relu","tanh"])
                   ))
    # Output Layer
    model.add(Dense(units=1, activation='sigmoid'))

    # define values for learning rate
    learning_rate = hp.Float("learning_rate",min_value = 0.001, max_value=0.1, step=0.01)

    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss = 'binary_crossentropy',
                  metrics = ['accuracy']
                 )
    return model

In [ ]:
import keras_tuner as kt

build_model(kt.HyperParameters())

In [ ]:
rtuner = kt.RandomSearch(hypermodel=build_model,
    objective="val_accuracy",
    max_trials=5,
    seed=12345,
    overwrite=True,
    directory="my_dir",
    project_name="diab")

In [ ]:
rtuner.search(X_train_scaled,
              y_train,
              epochs=10,
              validation_data=(X_test_scaled, y_test))

In [ ]:
models = rtuner.get_best_models(num_models=2)

In [ ]:
models[0].summary()

In [ ]:
models[1].summary()

In [ ]:
y_test_pred =  models[0].predict(X_test_scaled) >= 0.5

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(y_pred = y_test_pred, y_true = y_test )

In [ ]:
rtuner.get_best_hyperparameters()[0].get_config()['values']

In [ ]:
m1 = models[0]

### Explainable AI

In [ ]:
!pip install shap==0.37

In [ ]:
import shap

In [ ]:
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

In [ ]:
explainer = shap.KernelExplainer(m1, data=X_train_scaled)

In [ ]:
features = df_diab.columns

In [ ]:
features = features[0:-1]

In [ ]:
features

In [ ]:
df_diab.columns

In [ ]:
shap_values = explainer.shap_values(X_test_scaled)

In [ ]:
shap.summary_plot(shap_values, X_test_scaled, feature_names = features)

In [ ]:
values = shap_values[0]
base_values = [explainer.expected_value[0]]*len(shap_values[0])

tmp = shap.Explanation(values = np.array(values, dtype=np.float32),
                       base_values = np.array(base_values, dtype=np.float32),
                       data=np.array(X_test_scaled),
                       feature_names=features)

shap.plots.waterfall(tmp[0])